# Description:

I use a Garmin fenix watch and a set of sensors to collect data from my bike rides.

I analyse the data to find patterns etc.

One of the problems was that I could not link the cadence with the gear combinations, so I could not measure the power improvements.

This project tries to calucate the distance traveled by each revolution.

## Instructions:

Run the structure.sql and action.sql before running this notebook.

## TO DO:

1. Link with the cadence and speed data from Garmin to find the gear combination that was used;
2. Correct with the elevation to verify if the calcualtion is correct.

In [23]:
SELECT    chainring_cogs,
          sprocket_cogs,
          ratio,
          cycling.public.chainrings.name AS chainring_gear,
          cycling.public.sprockets.name AS sprocket_gear
FROM      cycling.public.v_gear_ratios
JOIN      cycling.public.chainrings ON cycling.public.chainrings.cogs = cycling.public.v_gear_ratios.chainring_cogs
JOIN      cycling.public.sprockets ON cycling.public.sprockets.cogs = cycling.public.v_gear_ratios.sprocket_cogs
LIMIT     1000

(27 row(s) affected)

Total execution time: 00:00:00.009

gear_id,chainring_cogs,sprocket_cogs,ratio,chainring_gear,sprocket_gear
4436,44,36,1.22,third,first
4432,44,32,1.38,third,second
4428,44,28,1.57,third,third
4424,44,24,1.83,third,fourth
4421,44,21,2.10,third,fifth
4418,44,18,2.44,third,sixth
4415,44,15,2.93,third,seventh
4413,44,13,3.38,third,eight
4411,44,11,4.00,third,nineth
3236,32,36,0.89,second,first


I noticed that there is a combination that does not work as the chain bends too much. This needs to be logged and excluded in the calcualtions.

The name of the gears are concatenated.

The cogs are the sum of the multiplication of <span style="color: rgb(33, 33, 33); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">chainrings cogs by 100 and the </span> <span style="color: rgb(33, 33, 33); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">sprockets cogs.</span>

In [22]:
DROP TABLE IF EXISTS public.impossible_combination cascade;
CREATE TABLE public.impossible_combination (
	id int GENERATED BY DEFAULT AS IDENTITY,
	"name" varchar NULL,
	"cogs" numeric NULL
);

WITH      chainrings AS (
          SELECT    "name",
                    cogs * 100 AS cogs_multiplier
          FROM      public.chainrings
          WHERE     cogs = 44
          ),
          sprockets AS (
          SELECT    "name",
                    cogs
          FROM      public.sprockets
          WHERE     cogs = 36
          )
INSERT    INTO public.impossible_combination ("name", cogs)
SELECT    chainrings.name || '-' || sprockets.name AS combined_names,
          chainrings.cogs_multiplier +             sprockets.cogs
FROM      chainrings,
          sprockets;
          
SELECT    *
FROM      public.impossible_combination;

DROP TABLE

CREATE TABLE

INSERT 0 1

(1 row(s) affected)

Total execution time: 00:00:00.037

id,name,cogs
1,third-first,4436


Finally I need to add <span style="color: rgb(33, 33, 33); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;"><b>gear_id</b> to filter out the impossible combinations.</span>

In [24]:
SELECT    chainring_cogs * 100 + sprocket_cogs AS gear_id,
          chainring_cogs,
          sprocket_cogs,
          ratio,
          cycling.public.chainrings.name       AS chainring_gear,
          cycling.public.sprockets.name        AS sprocket_gear
FROM      cycling.public.v_gear_ratios
JOIN      cycling.public.chainrings ON cycling.public.chainrings.cogs = cycling.public.v_gear_ratios.chainring_cogs
JOIN      cycling.public.sprockets ON cycling.public.sprockets.cogs = cycling.public.v_gear_ratios.sprocket_cogs
WHERE     (chainring_cogs * 100 + sprocket_cogs) NOT IN (
          SELECT    cogs
          FROM      public.impossible_combination
          )
LIMIT     1000

(26 row(s) affected)

Total execution time: 00:00:00.010

gear_id,chainring_cogs,sprocket_cogs,ratio,chainring_gear,sprocket_gear
4432,44,32,1.38,third,second
4428,44,28,1.57,third,third
4424,44,24,1.83,third,fourth
4421,44,21,2.10,third,fifth
4418,44,18,2.44,third,sixth
4415,44,15,2.93,third,seventh
4413,44,13,3.38,third,eight
4411,44,11,4.00,third,nineth
3236,32,36,0.89,second,first
3232,32,32,1.00,second,second


To find the top gear find the combination of the **<span style="color: rgb(33, 33, 33); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">chainring</span>**  <span style="color: var(--vscode-foreground);">with most cogs and with&nbsp;</span>   <span style="color: rgb(33, 33, 33); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;"><b>sprocket </b></span> <span style="color: var(--vscode-foreground);">least cogs.</span>

<span style="color: var(--vscode-foreground);">Or the combination that gives the <b>highest </b>ratio.</span>

In [8]:
SELECT    chainring_cogs,
          sprocket_cogs,
          ratio
FROM      cycling.public.v_gear_ratios
WHERE     chainring_cogs = (
          SELECT    MAX(chainring_cogs)
          FROM      cycling.public.v_gear_ratios
          )
AND       sprocket_cogs = (
          SELECT    MIN(sprocket_cogs)
          FROM      cycling.public.v_gear_ratios
          )
LIMIT     1000

(1 row(s) affected)

Total execution time: 00:00:01.348

chainring_cogs,sprocket_cogs,ratio
44,11,4.00


To find the bottom gear find the combination of the **chainring** with least cogs and with **sprocket** mosth cogs.

Or the combination that gives the **lowest** ratio.

In [12]:
SELECT    chainring_cogs,
          sprocket_cogs,
          ratio
FROM      cycling.public.v_gear_ratios
WHERE     chainring_cogs = (
          SELECT    MIN(chainring_cogs)
          FROM      cycling.public.v_gear_ratios
          )
AND       sprocket_cogs = (
          SELECT    MAX(sprocket_cogs)
          FROM      cycling.public.v_gear_ratios
          )
LIMIT     1000

(1 row(s) affected)

Total execution time: 00:00:01.346

chainring_cogs,sprocket_cogs,ratio
22,36,0.61


Now I need to filter out the impossible gear combinations.

I have the distance per one revolution (cadence 1).

With this I can multiply the <span style="color: rgb(33, 33, 33); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;"><b>distance_per_revolution</b> or </span> <span style="color: rgb(33, 33, 33); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;"><b>distance_per_revolution_cm </b></span> <span style="color: var(--vscode-foreground);">by the cadence from the Garmin sensor and I will get the distance in in/cm per minute.</span>

<span style="color: var(--vscode-foreground);">A few more calculations are required to get the result in km/h.</span>

In [28]:
SELECT    diameter,
          chainring_cogs * 100 + sprocket_cogs AS gear_id,
          ratio,
          chainring_cogs,
          sprocket_cogs,
          gear_inches,
          distance_per_revolution,
          distance_per_revolution_cm
FROM      public.v_distance_per_revolution
WHERE     (chainring_cogs * 100 + sprocket_cogs) NOT IN (
          SELECT    cogs
          FROM      public.impossible_combination
          )
LIMIT     1000

(26 row(s) affected)

Total execution time: 00:00:00.009

diameter,gear_id,ratio,chainring_cogs,sprocket_cogs,gear_inches,distance_per_revolution,distance_per_revolution_cm
27.5,4432,1.38,44,32,37.950,119.16300,302.67
27.5,4428,1.57,44,28,43.175,135.56950,344.35
27.5,4424,1.83,44,24,50.325,158.02050,401.37
27.5,4421,2.10,44,21,57.750,181.33500,460.59
27.5,4418,2.44,44,18,67.100,210.69400,535.16
27.5,4415,2.93,44,15,80.575,253.00550,642.63
27.5,4413,3.38,44,13,92.950,291.86300,741.33
27.5,4411,4.00,44,11,110.000,345.40000,877.32
27.5,3236,0.89,32,36,24.475,76.85150,195.20
27.5,3232,1.00,32,32,27.500,86.35000,219.33


Assume cadence is 1 (per minute), then the distance per hour wil be:

In [62]:
SELECT    diameter,
          chainring_cogs * 100 + sprocket_cogs AS gear_id,
          ratio,
          chainring_cogs,
          sprocket_cogs,
          gear_inches,
          distance_per_revolution as distance_per_revolution_in,
          distance_per_revolution_cm,
          round(distance_per_revolution / 633.60 * 60 /*cadence*/,2) as distance_mph,
          round(distance_per_revolution_cm / 1000 * 60 /*cadence*/,2) as distance_kmh
FROM      public.v_distance_per_revolution
WHERE     (chainring_cogs * 100 + sprocket_cogs) NOT IN (
          SELECT    cogs
          FROM      public.impossible_combination
          )
LIMIT     1000

(26 row(s) affected)

Total execution time: 00:00:00.009

diameter,gear_id,ratio,chainring_cogs,sprocket_cogs,gear_inches,distance_per_revolution_in,distance_per_revolution_cm,distance_mph,distance_kmh
27.5,4432,1.38,44,32,37.950,119.16300,302.67,11.28,18.16
27.5,4428,1.57,44,28,43.175,135.56950,344.35,12.84,20.66
27.5,4424,1.83,44,24,50.325,158.02050,401.37,14.96,24.08
27.5,4421,2.10,44,21,57.750,181.33500,460.59,17.17,27.64
27.5,4418,2.44,44,18,67.100,210.69400,535.16,19.95,32.11
27.5,4415,2.93,44,15,80.575,253.00550,642.63,23.96,38.56
27.5,4413,3.38,44,13,92.950,291.86300,741.33,27.64,44.48
27.5,4411,4.00,44,11,110.000,345.40000,877.32,32.71,52.64
27.5,3236,0.89,32,36,24.475,76.85150,195.20,7.28,11.71
27.5,3232,1.00,32,32,27.500,86.35000,219.33,8.18,13.16
